# gen_build_costs and gen_build_predetermined

In [1]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''

###
# module issue
import os
import sys
module_path = os.path.abspath(os.getcwd() + '\\..')
if module_path not in sys.path:
    sys.path.append(module_path)
###

import pandas as pd
import numpy as np

from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings
)
from pathlib import Path

from math import isnan

In [2]:
'''
Import based on imports found in the notebooks located here:
Schivley Greg, PowerGenome, (2022), GitHub repository, 
    https://github.com/PowerGenome/PowerGenome/tree/master/notebooks
'''

cwd = Path.cwd()

settings_path = (
    cwd / "settings_TD.yml" 
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

pudl_engine, pudl_out, pg_engine = init_pudl_connection(
    freq="AS",
    start_year=min(settings.get("data_years")),
    end_year=max(settings.get("data_years")),
)

# check_settings(settings, pg_engine)

# read in data sources
* files created by PowerGenome (existing_gen, potential_build_year, all_gen)
* PUDL data
    - generators_eia860
    - generators_entity_eia
* EIA excel file
    - 3_1_Generator_Y2020

In [3]:
# read in csv outputs from PG
existing_gen = pd.read_csv('PG_output_csv/existing_gen_WECC.csv', index_col=0)
# new_gen = pd.read_csv('PG_output_csv/new_gen_WECC.csv', index_col=0)
#existing_variability = pd.read_csv('PG_output_csv/existing_variability_WECC.csv', index_col=0)
potential_build_yr = pd.read_csv('PG_output_csv/gc_units_model_WECC.csv', index_col=0)
all_gen = pd.read_csv('PG_output_csv/all_gen_WECC.csv', index_col=0)
# fuels = pd.read_csv('PG_output_csv/fuels_WECC.csv', index_col=0)
# fuel_prices = pd.read_csv('PG_output_csv/fuel_prices_WECC.csv', index_col=0)
#load_curves = pd.read_csv('PG_output_csv/load_curves_WECC.csv', index_col=0)

In [4]:
'''
Catalyst Cooperative. “Pudl Data Dictionary.” PUDL Data Dictionary - PUDL 0.5.0 Documentation, 
    https://catalystcoop-pudl.readthedocs.io/en/v0.5.0/data_dictionaries/pudl_db.html. 
'''
# pull in data from PUDL tables
generators_eia860 = pd.read_sql_table("generators_eia860", pudl_engine)

generators_entity_eia = pd.read_sql_table("generators_entity_eia", pudl_engine)

In [5]:
# create copies of PUDL tables and filter to relevant columns
pudl_gen = generators_eia860.copy()
pudl_gen = pudl_gen[['plant_id_eia','generator_id', 'operational_status',
                     'retirement_date', 'planned_retirement_date','current_planned_operating_date']] #'utility_id_eia',

pudl_gen_entity = generators_entity_eia.copy()
pudl_gen_entity = pudl_gen_entity[['plant_id_eia', 'generator_id', 'operating_date']]

In [6]:
'''
“U.S. Energy Information Administration - EIA - Independent Statistics and Analysis.” 
Form EIA-860 Detailed Data with Previous Form Data (EIA-860A/860B), 9 Sept. 2021, 
https://www.eia.gov/electricity/data/eia860/.

Used the 2020 zip folder and 3_1_Generator_Y2020 file
'''
# pull in eia_Generator_Y2020 (operable and proposed)
eia_Generator_Y2020 = pd.read_excel('3_1_Generator_Y2020.xlsx', sheet_name=0, header=1)
eia_Generator_Y2020_proposed = pd.read_excel('3_1_Generator_Y2020.xlsx', sheet_name=1, header=1)

In [7]:
# copy of operable eia_Generator_Y2020 and filter to relevant years
eia_Gen = eia_Generator_Y2020.copy()
eia_Gen = eia_Gen[['Utility ID','Utility Name','Plant Code','Plant Name', 'Generator ID',
                   'Operating Year', 'Planned Retirement Year']]
eia_Gen = eia_Gen[eia_Gen['Plant Code'].notna()]

# create identifier to connect to powergenome data
eia_Gen['plant_gen_id'] = eia_Gen['Plant Code'].astype(str) +'_'+ eia_Gen['Generator ID']

# copy of proposed eia_Generator_Y2020 and filter to relevant years
eia_Gen_prop = eia_Generator_Y2020_proposed.copy()
eia_Gen_prop = eia_Gen_prop[['Utility ID','Utility Name','Plant Code','Plant Name', 'Generator ID',
                   'Effective Year']]
eia_Gen_prop = eia_Gen_prop[eia_Gen_prop['Plant Code'].notna()]

# create identifier to connect to powergenome data
eia_Gen_prop['plant_gen_id'] = eia_Gen_prop['Plant Code'].astype(str) +'_'+ eia_Gen_prop['Generator ID']
# eia_Gen_prop.head()

In [8]:
# create copies of potential_build_yr (powergenome)
pg_build = potential_build_yr.copy()
pg_build = pg_build[['plant_id_eia', 'generator_id','unit_id_pudl',
                     'planned_retirement_date', 'operating_date', 'Operating Year','retirement_year']]

# helper functions to create dictionaries and unique identifiers to connect tables
* it might be faster to merge tables instead of using dictionaries.

In [9]:
def create_dict_plantgen(df, column):
    '''
    Create dictionary from two columns, removing na's beforehand
    {plant_gen_id: year}
    '''
    df = df[df[column].notna()]
    ids = df['plant_gen_id'].to_list()
    dates = df[column].to_list()
    dictionary = dict(zip(ids, dates))
    return dictionary

def create_dict_plantpudl(df, column):
    '''
    Create dictionary from two columns, removing na's beforehand
    {plant_pudl_id: year}
    '''
    df = df[df[column]!='nan']
    ids = df['plant_pudl_id'].to_list()
    dates = df[column].to_list()
    dictionary = dict(zip(ids, dates))
    return dictionary

def plant_dict(plantideia, dictionary):
    '''
    Take key from pandas column, return value from dictionary. Passing if not in dictionary.
    '''
    if plantideia in dictionary:
        return dictionary[plantideia]
    else:
        pass

def plant_gen_id(df):
    '''
    Create unique id for generator by combining plant_id_eia and generator_id
    '''
    df['plant_gen_id'] = df['plant_id_eia'].astype(str) +'_'+ df['generator_id'].astype(str)
    return df

def plant_pudl_id(df):
    '''
    Create unique id for generator by combining plant_id_eia and unit_pudl_id
    '''
    df['plant_pudl_id'] = df['plant_id_eia'].astype(str) +'_'+ df['unit_id_pudl'].astype(str)
    return df

# manual overrides for certain plants that could not be found elsewhere
* retirement ages were used as a last resort. Backed retirement age off the retirement date 

In [10]:
# found plant names from pd.read_sql_table("plants_entity_eia", pudl_engine)
# did a google search on those names to find build year
manual_build_yr = {166.0:1931, 1230.0:1963, 7456.0:2001, 10718.0:1985, 50034.0:1992, 50177.0:1980,
                   50281.0:1982, 50322.0:1985, 50513.0:1992, 50560.0:1986, 50820.0:1983, 54355.0:1993,
                   55043.0:1998, 55177.0:2001, 55734.0:2002, 58044.0:2012, 59551.0:2014, 59553.0:2014, 60611.0:2016,
                  1359:1896}
# manual updates based on eia excel file (leading 0s) {plant_gen_id}
plant_gen_manual = {'55168.0_1':2002, '55168.0_2':2002, '55168.0_3':2002, }
plant_gen_manual_proposed = {'57943.0_6':2021}
plant_gen_manual_retired = {'64206.0_2004':2004}

# dictionary of retirement ages, pulled from settings
retirement_ages = settings.get('retirement_ages')

In [11]:
# has plant information
# plants_entity_eia = pd.read_sql_table("plants_entity_eia", pudl_engine)

# how tables seems to tie together
* pg_build and pudl_gen have plant_id_eia and generator_id => use those as unique ids
* pg_build and all_gen have plant_id_eia and unit_id_pudl => use those as unique ids

In [12]:
# modify the tables by adding the unique identifies for the plants

# add in the plant+generator ids to pg_build and pudl tables (plant_id_eia + generator_id)
pudl_gen = plant_gen_id(pudl_gen)
pudl_gen_entity = plant_gen_id(pudl_gen_entity)
pg_build = plant_gen_id(pg_build)

# add in the plant+pudl id to the all_gen and pg_build tables (plant_id_eia + unit_pudl_id)
pg_build = plant_pudl_id(pg_build)
all_gen = plant_pudl_id(all_gen)

# gen_build_predetermined

In [13]:
def gen_build_predetermined(all_gen, pudl_gen, pudl_gen_entity, pg_build, manual_build_yr, 
                            eia_Gen, eia_Gen_prop, plant_gen_manual, plant_gen_manual_proposed, 
                            plant_gen_manual_retired, retirement_ages):
    '''
    Create the gen_build_predetermined table
    Inputs
        1) all_gen: from PowerGenome gc.create_all_generators()
        2) pudl_gen: from PUDL generators_eia860
            - retirement_date
            - planned_retirement)date
            - current_planned_operating_date
        3) pudl_gen_entity: from PUDL generators_entity_eia
            - operating_date
        4) pg_build: from PowerGenome gc.units_model
            - planned_retirement_date
            - operating_date
            - Operating Year
            - retirement_year
        5) manual_build_yr: dictionary of build years that were found manually (outside of PUDL and PG)
        6) eia_Gen: eia operable plants
        7) eia_Gen_prop: eia proposed plants
        8) plant_gen_manual, plant_gen_manual_proposed, plant_gen_manual_retired: manually found build_years
        9) retirement_ages: how many years until plant retires
    Output columns
        * GENERATION_PROJECT: index from all_gen
        * build_year: using pudl_gen, pudl_gen_entity, eia excel file, and pg_build to get years
        * gen_predetermined_cap: based on Cap_size from all_gen
        * gen_predetermined_storage_energy_mwh: based on capex_mwh from all_gen
    Outputs
        gen_buildpre: is the 'offical' table
        gen_build_with_id: is gen_buildpre before 2020 was taken out and with plant_id in it
    
    '''
    
    '''
    Use dictionaries to get the build year from the various sources of information
    '''
    
    # create dictionaries {plant_gen_id: date} from pudl_gen
    plant_op_date_dict = create_dict_plantgen(pudl_gen, 'current_planned_operating_date')
    plant_plan_ret_date_dict = create_dict_plantgen(pudl_gen, 'planned_retirement_date')
    plant_ret_date_dict = create_dict_plantgen(pudl_gen,'retirement_date')

    # create dictionaries {plant_gen_id: date} from pudl_gen_entity
    entity_op_date_dict = create_dict_plantgen(pudl_gen_entity, 'operating_date')

    # create dictionaries {plant_gen_id: date} from pg_build
    PG_pl_retire_date_dict = create_dict_plantgen(pg_build, 'planned_retirement_date')
    PG_retire_yr_dict = create_dict_plantgen(pg_build, 'retirement_year')
    PG_op_date_dict = create_dict_plantgen(pg_build, 'operating_date')
    PG_op_yr_dict = create_dict_plantgen(pg_build,'Operating Year')
    
    #  create dictionaries {plant_gen_id: date} from eia excel file
    eia_Gen_dict = create_dict_plantgen(eia_Gen, 'Operating Year')
    eia_Gen_prop_dict = create_dict_plantgen(eia_Gen_prop, 'Effective Year')
    
    '''
    Bring in dates based on dictionaries and the plant_gen_id column
    '''
    # based on pudl_gen
    pg_build['op_date'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, plant_op_date_dict))
    pg_build['plan_retire_date'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, 
                                                                                        plant_plan_ret_date_dict))
    pg_build['retirement_date'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, plant_ret_date_dict))

    # based on pudl_gen_entity
    pg_build['entity_op_date'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, entity_op_date_dict))

    # based on pg_build
    pg_build['PG_pl_retire'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, PG_pl_retire_date_dict))
    pg_build['PG_retire_yr'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, PG_retire_yr_dict))
    pg_build['PG_op_date'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, PG_op_date_dict))
    pg_build['PG_op_yr'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, PG_op_yr_dict))

    # based on manual_build
    pg_build['manual_yr'] = pg_build['plant_id_eia'].apply(lambda x: plant_dict(x, manual_build_yr))

    # based on eia excel
    pg_build['eia_gen_op_yr'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, eia_Gen_dict))
    pg_build['proposed_year'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, eia_Gen_prop_dict))
    
    # based on eia excel manual dictionary
    pg_build['eia_gen_manual_yr'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, plant_gen_manual))
    pg_build['proposed_manual_year'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, 
                                                                                    plant_gen_manual_proposed))
    pg_build['eia_gen_retired_yr'] = pg_build['plant_gen_id'].apply(lambda x: plant_dict(x, 
                                                                                    plant_gen_manual_retired))
    
    '''
    Manipulating the build and retirement year data
        - change to year instead of date, 
        - bring all years into one column
        - remove nans
    '''
    
    # the columns that have the dates as datetime
    columns = ['operating_date', 'op_date', 'plan_retire_date', 'retirement_date', 'entity_op_date', 
               'planned_retirement_date', 'PG_pl_retire', 'PG_op_date']
    # change those columns to just year (instead of longer date)
    for c in columns:
        try:
            pg_build[c] = pd.DatetimeIndex(pg_build[c]).year.astype(str)
        except:
            pass

    # get all build years into one column (includes manual dates and proposed dates)
    pg_build['yr'] = pg_build['op_date'] +','+ pg_build['entity_op_date'].astype(str)  +','+  pg_build[
        'operating_date']  +','+  pg_build['Operating Year'].astype(str) +','+ pg_build['manual_yr'].astype(str
                            ) +','+ pg_build['PG_op_date'].astype(str) +','+ pg_build['PG_op_yr'].astype(str
                        ) +','+ pg_build['eia_gen_op_yr'].astype(str) +','+ pg_build['eia_gen_manual_yr'
                        ].astype(str) +','+ pg_build['proposed_year'].astype(str) +','+ pg_build[
                                                        'proposed_manual_year'].astype(str)

    # remove nans from combined build year lists
    pg_build['yr'] = pg_build['yr'].str.replace('nan,','')
    pg_build['yr'] = pg_build['yr'].str.replace(',nan','')
    pg_build['yr'] = pg_build['yr'].str.replace(',None','')
    pg_build['yr'] = pg_build['yr'].str.replace('None,','')

    # unique years for build year
    year = pg_build[['yr']]
    year_list = year.values.tolist()
    for i in range(len(year_list)):
        year_list[i] = year_list[i][0].split(',')
        year_list[i] = list(set(year_list[i]))[0]
    pg_build['build_final'] = year_list


    # get all retirement years into one column
    pg_build['retirement'] = pg_build['planned_retirement_date'] +','+ pg_build[
        'retirement_date'] +','+ pg_build['plan_retire_date'] +','+ pg_build['retirement_year'].astype(str
                    ) +','+ pg_build['PG_pl_retire'] +','+ pg_build['PG_retire_yr'].astype(str
                    ) +','+ pg_build['eia_gen_retired_yr'].astype(str)

    # remve nans from combined retirement year lists
    pg_build['retirement'] = pg_build['retirement'].str.replace('nan,','')
    pg_build['retirement'] = pg_build['retirement'].str.replace(',nan','')
    pg_build['retirement'] = pg_build['retirement'].str.replace(',None','')
    pg_build['retirement'] = pg_build['retirement'].str.replace('None,','')

    # pick latest retirement year
    retire = pg_build[['retirement']]
    retire_list = retire.values.tolist()
    for i in range(len(retire_list)):
        retire_list[i] = retire_list[i][0].split(',')
        adjusted_year = max(retire_list[i])
        retire_list[i] = adjusted_year
    pg_build['retire_year_final'] = retire_list

    '''
    Start creating the gen_build_predetermined table
    '''
    # base it off of PowerGenome all_gen
    gen_buildpre = all_gen.copy()
    gen_buildpre = gen_buildpre[['index','plant_id_eia', 'Cap_size', 'capex_mwh','region', 'plant_pudl_id',
                                 'technology']]   
    
    # based GENERATION_PROJECT off of the index of all_gen
    gen_buildpre['GENERATION_PROJECT'] = gen_buildpre.index+1
    
    # this ignores new builds
    new_builds = gen_buildpre[gen_buildpre['index'].isna()]
    gen_buildpre = gen_buildpre[gen_buildpre['index'].notna()]
    
    # create dictionary to go from pg_build to gen_buildpre (build_year)
    pg_build_buildyr = create_dict_plantpudl(pg_build,'build_final')
    gen_buildpre['build_year'] = gen_buildpre['plant_pudl_id'].apply(lambda x: plant_dict(x, pg_build_buildyr))

    # create dictionary to go from pg_build to gen_buildpre (retirement_year)
    pg_retireyr = pg_build['retire_year_final'].to_list()
    pg_build_retireyr = create_dict_plantpudl(pg_build,'retire_year_final')
    gen_buildpre['retirement_year'] = gen_buildpre['plant_pudl_id'].apply(lambda x: plant_dict(x, 
                                                                                            pg_build_retireyr))    
    
    
    # for plants that still don't have a build year but have a retirement year. 
    # Base build year off of retirement year: retirement year - retirement age (based on technology)
    # check to see if it is na or None if you get blank build years
    mask = (gen_buildpre['build_year']=='None')
    nans = gen_buildpre[mask]

    gen_buildpre.loc[mask, 'build_year'] = nans.apply(lambda row: float(row.retirement_year) - 
                                              retirement_ages[row.technology], axis=1)
    
    # don't include new builds in gen_build_predetermined
#     new_builds['GENERATION_PROJECT'] = range(gen_buildpre.shape[0]+1, gen_buildpre.shape[0]+1+new_builds.shape[0])
#     new_builds = new_builds[['GENERATION_PROJECT', 'Cap_size', 'capex_mwh']]
#     new_builds2020 = new_builds.copy()
#     new_builds2030 = new_builds.copy()
#     new_builds2040 = new_builds.copy()
#     new_builds2050 = new_builds.copy()
#     new_builds2020['build_year'] = 2020
#     new_builds2030['build_year'] = 2030
#     new_builds2040['build_year'] = 2040
#     new_builds2050['build_year'] = 2050
    
    # filter to final columns
    # gen_build_with_id is an unmodified version of gen_build_pre (still has 2020 plant years)
    gen_build_with_id = gen_buildpre.copy()
    gen_build_with_id = gen_buildpre[['GENERATION_PROJECT', 'build_year','plant_id_eia','retirement_year', 
                                      'plant_pudl_id','technology']] # this table is for comparison/testing only
    gen_buildpre = gen_buildpre[['GENERATION_PROJECT', 'build_year', 'Cap_size', 'capex_mwh']]
    
    # don't include new builds
#     gen_buildpre_combined = pd.concat([gen_buildpre, new_builds2020, new_builds2030, new_builds2040, new_builds2050],
#                                      ignore_index=True)
#     gen_buildpre = gen_buildpre.append([new_builds2020, new_builds2030, new_builds2040, new_builds2050], 
#                                        ignore_index=True)
    
    
    gen_buildpre.rename(columns={'Cap_size':'gen_predetermined_cap', 
                                'capex_mwh': 'gen_predetermined_storage_energy_mwh'}, inplace=True) 
    # based on REAM
    gen_buildpre['gen_predetermined_storage_energy_mwh'] = gen_buildpre[
                                'gen_predetermined_storage_energy_mwh'].fillna('.')
    
    gen_buildpre['build_year'] = gen_buildpre['build_year'].astype(float).astype(int)
#     gen_buildpre['GENERATION_PROJECT'] = gen_buildpre['GENERATION_PROJECT'].astype(str)
    
    # SWITCH doesn't like having build years that are in the period
    gen_buildpre.drop(gen_buildpre[gen_buildpre['build_year']==2020].index, inplace = True)
    
    return gen_buildpre, gen_build_with_id

In [14]:
gen_buildpre, gen_build_with_id = gen_build_predetermined(all_gen, pudl_gen, pudl_gen_entity, 
                            pg_build, manual_build_yr, eia_Gen, eia_Gen_prop, plant_gen_manual, 
                            plant_gen_manual_proposed, plant_gen_manual_retired, retirement_ages)

In [15]:
gen_buildpre

,GENERATION_PROJECT,build_year,gen_predetermined_cap,gen_predetermined_storage_energy_mwh
0,1,2004,1.333,.
1,2,2008,11.839,.
2,3,2012,0.853,.
3,4,2012,0.853,.
4,5,1981,1.400,.
...,...,...,...,...
4347,4348,2018,1.200,.
4348,4349,2018,0.800,.
4349,4350,2021,2.900,.
4350,4351,2021,1.800,.


In [16]:
# check for blanks
gen_buildpre[gen_buildpre['build_year']=='None']

,GENERATION_PROJECT,build_year,gen_predetermined_cap,gen_predetermined_storage_energy_mwh


In [17]:
# these are already retired and should be removed
retired = gen_build_with_id[gen_build_with_id['retirement_year']<'2021']
retired_ids = retired['GENERATION_PROJECT'].to_list()
retired_ids

[3225, 4070]

# gen_build_costs
* need to run SWITCH_genbuildcosts_helper first

In [18]:
build_yr_list = gen_build_with_id['build_year'].to_list()
# using gen_build_with_id because it has plants that were removed for the final gen_build_pred. (ie. build year=2020)
gen_project = gen_build_with_id['GENERATION_PROJECT'].to_list()
build_yr_plantid_dict = dict(zip(gen_project, build_yr_list))

In [19]:
# these csv files are created from the SWITCH_genbuildcosts_helper notebook
new_gen_2020 = pd.read_csv('new_gen_2020.csv', index_col=0)
new_gen_2030 = pd.read_csv('new_gen_2030.csv', index_col=0)
new_gen_2040 = pd.read_csv('new_gen_2040.csv', index_col=0)
new_gen_2050 = pd.read_csv('new_gen_2050.csv', index_col=0)

In [20]:
def gen_build_costs_table(existing_gen, new_gen_2020, new_gen_2030, new_gen_2040, new_gen_2050, 
                          build_yr_plantid_dict, all_gen):
    '''
    Create gen_build_costs table based off of REAM Scenarior 178.
    Inputs
        pandas dataframes
            existing_gen - from PowerGenome gc.create_region_technology_clusters()
            new_gen_2020 - created by the gen_build_costs notebook
            new_gen_2030 - created by the gen_build_costs notebook
            new_gen_2040 - created by the gen_build_costs notebook
            new_gen_2050 - created by the gen_build_costs notebook
            all_gen - created by PowerGenome
        build_yr_plantid_dict - maps {generation_project: build_year}
        
    Output columns
        * GENERATION_PROJECT: based on index
        * build_year: based off of the build years from gen_build_predetermined
        * gen_overnight_cost: is 0 for existing, and uses PG capex_mw values for new generators
        * gen_fixed_om: is 0 for existing, and uses PG Fixed_OM_Cost_per_MWyr *1000 (SWITCH is per KW) for new gen
        * gen_storage_energy_overnight_cost: is 0 for existing and uses PG capex_mwh for new generators
    '''
    
    existing = existing_gen.copy()
#     existing = existing[['index','plant_id_eia']]
    existing['GENERATION_PROJECT'] = existing.index +1
#     existing['GENERATION_PROJECT'] = existing['GENERATION_PROJECT'].astype(str)
    existing['build_year'] = existing['GENERATION_PROJECT'].apply(lambda x: build_yr_plantid_dict[x])
    existing['gen_overnight_cost'] = 0
    existing['gen_fixed_om'] = 0
    existing['gen_storage_energy_overnight_cost'] = 0
    existing = existing[['GENERATION_PROJECT', 'build_year', 'gen_overnight_cost', 'gen_fixed_om',
                         'gen_storage_energy_overnight_cost']]
#     existing.rename(columns={'index':'GENERATION_PROJECT'}, inplace=True)
#     existing.drop('plant_id_eia',axis=1, inplace=True)
    
    
    combined_new_gens = pd.DataFrame()
    df_list = [new_gen_2020, new_gen_2030, new_gen_2040, new_gen_2050]
    year_list = [2020, 2030, 2040, 2050]
    for i in range(len(year_list)):
        df = df_list[i]
        df['build_year'] = year_list[i]
        # start the new GENERATION_PROJECT ids from the end of existing_gen (should tie out to same as gen_proj_info)
        df['GENERATION_PROJECT'] = range(existing.shape[0]+1, existing.shape[0]+1+df.shape[0])
        df['GENERATION_PROJECT'] = df['GENERATION_PROJECT'].astype(str)
        combined_new_gens = combined_new_gens.append(df)

    combined_new_gens['gen_fixed_om'] = combined_new_gens['Fixed_OM_Cost_per_MWyr'].apply(lambda x: x*1000)
    combined_new_gens.drop('Fixed_OM_Cost_per_MWyr',axis=1,inplace=True)
    combined_new_gens.rename(columns={'capex_mw':'gen_overnight_cost','capex_mwh':'gen_storage_energy_overnight_cost'},
                        inplace=True)
    
    combined_new_gens = combined_new_gens[['GENERATION_PROJECT','build_year','gen_overnight_cost', 'gen_fixed_om',
                                           'gen_storage_energy_overnight_cost']]
    

    gen_build_costs = existing.append(combined_new_gens, ignore_index=True)
    
    gen_build_costs['build_year'] = gen_build_costs['build_year'].astype(float).astype(int)
#     gen_build_costs.drop('index', axis=1, inplace=True)
    
    
    # gen_storage_energy_overnight_cost should only be for batteries
    all_gen['GP'] = all_gen.index+1
    batteries = all_gen[all_gen['technology']=='Battery_*_Moderate']
    batteries_id = batteries['GP'].to_list()
#     gen_build_costs['gen_storage_energy_overnight_cost'] = gen_build_costs.apply(
#                 lambda row: row.gen_storage_energy_overnight_cost if row.GENERATION_PROJECT in 
#                 batteries_id else '.',  axis=1)
    gen_build_costs['GENERATION_PROJECT'] = gen_build_costs['GENERATION_PROJECT'].astype(int)
    gen_build_costs.loc[~gen_build_costs['GENERATION_PROJECT'].isin(batteries_id),
                        'gen_storage_energy_overnight_cost'] = '.'
    
    return gen_build_costs

In [21]:
gen_build_costs = gen_build_costs_table(existing_gen, new_gen_2020, new_gen_2030, new_gen_2040, new_gen_2050, 
                                        build_yr_plantid_dict, all_gen)
gen_build_costs

,GENERATION_PROJECT,build_year,gen_overnight_cost,gen_fixed_om,gen_storage_energy_overnight_cost
0,1,2004,0.000000e+00,0,.
1,2,2008,0.000000e+00,0,.
2,3,2012,0.000000e+00,0,.
3,4,2012,0.000000e+00,0,.
4,5,1981,0.000000e+00,0,.
...,...,...,...,...,...
4668,4429,2050,1.962712e+06,62416000,.
4669,4430,2050,8.693962e+05,26174000,.
4670,4431,2050,7.211678e+05,20038000,.
4671,4432,2050,1.739989e+05,4349000,97458.020465


# Remove retired plants

In [22]:
# drop retired plants
gen_build_costs.drop(gen_build_costs[gen_build_costs['GENERATION_PROJECT'].isin(retired_ids)].index, inplace = True)
# drop retired plants
gen_buildpre.drop(gen_buildpre[gen_buildpre['GENERATION_PROJECT'].isin(retired_ids)].index, inplace = True)

In [23]:
gen_buildpre

,GENERATION_PROJECT,build_year,gen_predetermined_cap,gen_predetermined_storage_energy_mwh
0,1,2004,1.333,.
1,2,2008,11.839,.
2,3,2012,0.853,.
3,4,2012,0.853,.
4,5,1981,1.400,.
...,...,...,...,...
4347,4348,2018,1.200,.
4348,4349,2018,0.800,.
4349,4350,2021,2.900,.
4350,4351,2021,1.800,.


In [24]:
gen_build_costs

,GENERATION_PROJECT,build_year,gen_overnight_cost,gen_fixed_om,gen_storage_energy_overnight_cost
0,1,2004,0.000000e+00,0,.
1,2,2008,0.000000e+00,0,.
2,3,2012,0.000000e+00,0,.
3,4,2012,0.000000e+00,0,.
4,5,1981,0.000000e+00,0,.
...,...,...,...,...,...
4668,4429,2050,1.962712e+06,62416000,.
4669,4430,2050,8.693962e+05,26174000,.
4670,4431,2050,7.211678e+05,20038000,.
4671,4432,2050,1.739989e+05,4349000,97458.020465


In [25]:
gen_buildpre.to_csv(r'SWITCH_Inputs\gen_build_predetermined.csv', index = False)
gen_build_costs.to_csv(r'SWITCH_Inputs\gen_build_costs.csv', index = False)